In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [19]:
filename = "data.csv"
data = pd.read_csv(filename, index_col = 'A')
data

,C1,C2
A,,
A1,3.0,9.0
A2,7.0,7.0
A3,9.0,2.0


In [33]:
def sample_weights(n: int, sum_to = 1.0) -> list:
    q_j = np.random.uniform(0, 1, n-1) * sum_to

    q_j = np.sort(q_j)
    q_j = np.insert(q_j, 0, 0)
    q_j = np.insert(q_j, n, sum_to)

    w = [q_j[i]-q_j[i-1] for i in range(1, n+1)]
    return w

np.sum(sample_weights(10, 0.5))

0.5

In [6]:
# def generate_random_weight_vector(data):
#     num_elements = len(data)
#     weights = np.random.choice(data, num_elements, replace=False)
#     weights = weights / np.sum(weights)
#     return weights
#
# data = [0.2, 0.3, 0.5, 0.1]
# weight_vector = generate_random_weight_vector(data)
# print(weight_vector)

[0.18181818 0.45454545 0.09090909 0.27272727]


In [4]:
def compute_partial_utility(vector):
    min_value = np.min(vector)
    max_value = np.max(vector) - min_value
    return (vector - min_value) / max_value

In [27]:
def partial_utility_matrix(matrix):
    m, n = matrix.shape
    X = matrix.copy().to_numpy()
    for col in range(n):
        criterium = X[:, col]
        utility_vector = compute_partial_utility(criterium)
        X[:, col] = utility_vector
    return X
    # for col in matrix:
    #     criterium = matrix[col]
    #     utility_vector = compute_partial_utility(criterium.tolist())
    #     utility_vector = dict(zip(criterium.index, utility_vector))
    #     X[col] = pd.Series(utility_vector)
    # return X

partial_utility_matrix(data)
data

,C1,C2
A,,
A1,3.0,9.0
A2,7.0,7.0
A3,9.0,2.0


In [6]:
def utility(x_i, w):
    return np.sum(x_i.dot(w))

In [34]:
import time
from numba import njit

def monte_carlo_simulation(it, m, n, X_utility):
     w_c = np.zeros([m,n])
     h = np.zeros([m,m])
     for i in tqdm(range(it)):
        w = sample_weights(n)
        t = np.array([utility(X_utility[j,:], w) for j in range(m)])
        rank =  np.argsort(t)
        for j in range(m):
            r = rank[j]
            h[r,j] += 1
            if r == (m-1):
                w_c[j] += w
     return h, w_c


def alg_3(X, it: int = 10):
    m, n = X.shape
    X_utility = partial_utility_matrix(X)

    h, w_c = monte_carlo_simulation(it, m, n, X_utility)
    b = np.zeros([m,m])
    for i in range(m):
        if h[m-1][i] > 0:
            w_c[i] = w_c[i] / h[m-1,i]
        for j in range(m):
            b[j,i] = h[j,i] / it

    return w_c, b

def alg_4(X, b, it):
    m, n = X.shape
    X_utility = partial_utility_matrix(X)
    p = np.zeros(m)

    for i in range(it):
        pass
    return NotImplemented

K_w = 10000
K_c = 10
t = time.time()
w_c, b = alg_3(data, it=K_w)
print(f"Time: {(time.time()-t)}")
df_text = pd.DataFrame(b)
print(b)
print(w_c)
print(np.sum(w_c, axis=1))

100%|██████████| 10000/10000 [00:00<00:00, 10102.95it/s]

Time: 0.9945671558380127
[[0.4971 0.2021 0.3008]
 [0.     0.6163 0.3837]
 [0.5029 0.1816 0.3155]]
[[0.25361817 0.74638183]
 [0.59072634 0.40927366]
 [0.84161841 0.15838159]]
[1. 1. 1.]
